<a href="https://colab.research.google.com/github/Abhinav-meluha/Last-mile-Delivery-/blob/main/Untitled11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import streamlit as st

In [ ]:
file_path = "/content/drive/MyDrive/Copy of Last mile Delivery Data.csv"
df = pd.read_csv(file_path)
df.head()


,Order_ID,Agent_Age,Agent_Rating,Store_Latitude,Store_Longitude,Drop_Latitude,Drop_Longitude,Order_Date,Order_Time,Pickup_Time,Weather,Traffic,Vehicle,Area,Delivery_Time,Category
0,ialx566343618,37,4.9,22.745049,75.892471,22.765049,75.912471,2022-03-19,11:30:00,11:45:00,Sunny,High,motorcycle,Urban,120,Clothing
1,akqg208421122,34,4.5,12.913041,77.683237,13.043041,77.813237,2022-03-25,19:45:00,19:50:00,Stormy,Jam,scooter,Metropolitian,165,Electronics
2,njpu434582536,23,4.4,12.914264,77.678400,12.924264,77.688400,2022-03-19,08:30:00,08:45:00,Sandstorms,Low,motorcycle,Urban,130,Sports
3,rjto796129700,38,4.7,11.003669,76.976494,11.053669,77.026494,2022-04-05,18:00:00,18:10:00,Sunny,Medium,motorcycle,Metropolitian,105,Cosmetics
4,zguw716275638,32,4.6,12.972793,80.249982,13.012793,80.289982,2022-03-26,13:30:00,13:45:00,Cloudy,High,scooter,Metropolitian,150,Toys


In [ ]:
df = df.rename(columns={
    "Delivery Time": "delivery_time",
    "Traffic": "traffic",
    "Weather": "weather",
    "Vehicle Type": "vehicle",
    "Agent Age": "agent_age",
    "Agent Rating": "agent_rating",
    "Area": "area",
    "Category": "category"
}, errors="ignore")

df.head()


,Order_ID,Agent_Age,Agent_Rating,Store_Latitude,Store_Longitude,Drop_Latitude,Drop_Longitude,Order_Date,Order_Time,Pickup_Time,weather,traffic,Vehicle,area,Delivery_Time,category
0,ialx566343618,37,4.9,22.745049,75.892471,22.765049,75.912471,2022-03-19,11:30:00,11:45:00,Sunny,High,motorcycle,Urban,120,Clothing
1,akqg208421122,34,4.5,12.913041,77.683237,13.043041,77.813237,2022-03-25,19:45:00,19:50:00,Stormy,Jam,scooter,Metropolitian,165,Electronics
2,njpu434582536,23,4.4,12.914264,77.678400,12.924264,77.688400,2022-03-19,08:30:00,08:45:00,Sandstorms,Low,motorcycle,Urban,130,Sports
3,rjto796129700,38,4.7,11.003669,76.976494,11.053669,77.026494,2022-04-05,18:00:00,18:10:00,Sunny,Medium,motorcycle,Metropolitian,105,Cosmetics
4,zguw716275638,32,4.6,12.972793,80.249982,13.012793,80.289982,2022-03-26,13:30:00,13:45:00,Cloudy,High,scooter,Metropolitian,150,Toys


In [ ]:
df.columns = df.columns.str.strip().str.lower().str.replace(" ", "_")
print(df.columns)


Index(['order_id', 'agent_age', 'agent_rating', 'store_latitude',
       'store_longitude', 'drop_latitude', 'drop_longitude', 'order_date',
       'order_time', 'pickup_time', 'weather', 'traffic', 'vehicle', 'area',
       'delivery_time', 'category'],
      dtype='object')


In [ ]:
print(df.columns.tolist())


['order_id', 'agent_age', 'agent_rating', 'store_latitude', 'store_longitude', 'drop_latitude', 'drop_longitude', 'order_date', 'order_time', 'pickup_time', 'weather', 'traffic', 'vehicle', 'area', 'delivery_time', 'category']


In [ ]:
df['delivery_time'] = pd.to_numeric(df['delivery_time'], errors='coerce')
df['agent_age'] = pd.to_numeric(df['agent_age'], errors='coerce')
df['agent_rating'] = pd.to_numeric(df['agent_rating'], errors='coerce')
print(df[['delivery_time', 'agent_age', 'agent_rating']].head())


   delivery_time  agent_age  agent_rating
0            120         37           4.9
1            165         34           4.5
2            130         23           4.4
3            105         38           4.7
4            150         32           4.6


In [ ]:
num_cols = ['delivery_time', 'agent_age', 'agent_rating']
cat_cols = ['weather', 'traffic', 'vehicle', 'area', 'category']

for col in num_cols:
    df[col] = df[col].fillna(df[col].median())

for col in cat_cols:
    df[col] = df[col].fillna(df[col].mode()[0])

# 🔥 SHOW OUTPUT HERE
print("Missing values after cleaning:")
print(df[num_cols + cat_cols].isnull().sum())


Missing values after cleaning:
delivery_time    0
agent_age        0
agent_rating     0
weather          0
traffic          0
vehicle          0
area             0
category         0
dtype: int64


In [ ]:
mean_dt = df['delivery_time'].mean()
std_dt = df['delivery_time'].std()
threshold = mean_dt + std_dt

df['late'] = df['delivery_time'] > threshold

print(df[['delivery_time', 'late']])

       delivery_time   late
0                120  False
1                165  False
2                130  False
3                105  False
4                150  False
...              ...    ...
43734            160  False
43735            180   True
43736             80  False
43737            130  False
43738            180   True

[43739 rows x 2 columns]


In [ ]:
def age_group(age):
    if age < 25:
        return "Under 25"
    elif 25 <= age <= 40:
        return "25-40"
    else:
        return "40+"

df['age_group'] = df['agent_age'].apply(age_group)


In [ ]:
delay_df = df.groupby(['weather', 'traffic'], as_index=False)['delivery_time'].mean()

fig_delay = px.bar(
    delay_df,
    x='traffic',
    y='delivery_time',
    color='weather',
    barmode='group',
    title='Average Delivery Time by Weather and Traffic'
)
fig_delay.show()


In [ ]:
vehicle_df = df.groupby('vehicle', as_index=False)['delivery_time'].mean()

fig_vehicle = px.bar(
    vehicle_df,
    x='vehicle',
    y='delivery_time',
    title='Average Delivery Time by Vehicle Type'
)
fig_vehicle.show()


In [ ]:
fig_agent = px.scatter(
    df,
    x='agent_rating',
    y='delivery_time',
    color='age_group',
    title='Agent Rating vs Delivery Time'
)
fig_agent.show()


In [ ]:
area_df = df.groupby('area', as_index=False)['delivery_time'].mean()

fig_area = px.bar(
    area_df,
    x='area',
    y='delivery_time',
    title='Average Delivery Time by Area'
)
fig_area.show()


In [ ]:
 fig_category = px.box(
    df,
    x='category',
    y='delivery_time',
    title='Delivery Time Distribution by Category'
)
fig_category.show()


In [ ]:
!pip install streamlit
!npm install -g localtunnel


⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏
changed 22 packages in 2s
⠏
⠏3 packages are looking for funding
⠏  run `npm fund` for details
⠏

In [ ]:
%%writefile app.py
import streamlit as st

st.title("Test Streamlit App")
st.write("If you can see this, the tunnel is working!")


Overwriting app.py


In [ ]:
!npx localtunnel --port 8501


⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴your url is: https://icy-rocks-slide.loca.lt
^C


In [ ]:
!streamlit run app.py --server.port 8501 --server.address 0.0.0.0 &>/content/streamlit.log &


In [ ]:
!npx localtunnel --port 8501


⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸your url is: https://tame-walls-enter.loca.lt
^C


In [ ]:
!pip install streamlit
!wget -q https://bin.equinox.io/c/bNyj1mQVY4c/ngrok-v3-stable-linux-amd64.tgz
!tar -xvzf ngrok-v3-stable-linux-amd64.tgz
!mv ngrok /usr/local/bin/ngrok
!ngrok version


ngrok
ngrok version 3.33.1


In [ ]:
!ngrok config add-authtoken 3639E7TGXClBKag7HKEwwbM8jTA_2BGw7fBpuKavd2e4QnUS7


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
%%writefile app.py
import streamlit as st
st.title("Test Streamlit + ngrok")
st.write("✅ If you see this page, ngrok tunnel works!")


Overwriting app.py


In [ ]:
!streamlit run app.py --server.port 8501 --server.address 0.0.0.0 &>/content/streamlit.log &


In [ ]:
!ngrok http 8501 & sleep 5 && curl -s http://localhost:4040/api/tunnels | grep -o "https:[^\\\"]*"


^C


In [ ]:
import os
from pyngrok import ngrok

# Force pyngrok to use our freshly installed ngrok v3
os.environ["NGROK_PATH"] = "/usr/local/bin/ngrok"

# Open a HTTP tunnel on port 8501
public_tunnel = ngrok.connect(8501, "http")
print("🌐 Your public Streamlit URL:", public_tunnel.public_url)


🌐 Your public Streamlit URL: https://unpuckered-unforgivingly-alex.ngrok-free.dev


In [ ]:
!pip install streamlit requests -q

# Download latest ngrok v3 binary
!wget -q https://bin.equinox.io/c/bNyj1mQVY4c/ngrok-v3-stable-linux-amd64.tgz
!tar -xvzf ngrok-v3-stable-linux-amd64.tgz
!mv ngrok /usr/local/bin/ngrok

# Verify ngrok version
!ngrok version


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 82.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 140.0 MB/s eta 0:00:00
ngrok
ngrok version 3.33.1


In [ ]:
%%writefile app.py
import streamlit as st

st.title("✅ Streamlit + ngrok test")
st.write("If you see this page in the browser, your tunnel works!")


Overwriting app.py


In [ ]:
!streamlit run app.py --server.port 8501 --server.address 0.0.0.0 &>/content/streamlit.log &


In [ ]:
NGROK_AUTH_TOKEN = "3639E7TGXClBKag7HKEwwbM8jTA_2BGw7fBpuKavd2e4QnUS7"

!ngrok config add-authtoken $NGROK_AUTH_TOKEN


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
%%writefile app.py
import streamlit as st
import pandas as pd
import numpy as np
import plotly.express as px

# 🔹 Path to your dataset on Drive
DATA_PATH = "/content/drive/MyDrive/Copy of Last mile Delivery Data.csv"

@st.cache_data
def load_data():
    df = pd.read_csv(DATA_PATH)

    # Clean column names
    df.columns = df.columns.str.strip().str.lower().str.replace(" ", "_")

    # Convert numeric columns
    df["delivery_time"] = pd.to_numeric(df["delivery_time"], errors="coerce")
    df["agent_age"] = pd.to_numeric(df["agent_age"], errors="coerce")
    df["agent_rating"] = pd.to_numeric(df["agent_rating"], errors="coerce")

    # Handle missing values
    num_cols = ["delivery_time", "agent_age", "agent_rating"]
    cat_cols = ["weather", "traffic", "vehicle", "area", "category"]

    for col in num_cols:
        df[col] = df[col].fillna(df[col].median())

    for col in cat_cols:
        df[col] = df[col].fillna(df[col].mode()[0])

    # Remove non-positive delivery times (if any)
    df = df[df["delivery_time"] > 0].copy()

    # Late delivery flag = > mean + 1 std
    mean_dt = df["delivery_time"].mean()
    std_dt = df["delivery_time"].std()
    threshold = mean_dt + std_dt
    df["late"] = df["delivery_time"] > threshold

    # Age groups
    def age_group(age):
        if age < 25:
            return "Under 25"
        elif age <= 40:
            return "25–40"
        else:
            return "40+"

    df["age_group"] = df["agent_age"].apply(age_group)

    return df

# ----------------- MAIN APP -----------------

df = load_data()

st.title("📦 Last-Mile Delivery Analytics Dashboard")

st.markdown(
    "Interactive dashboard built from last-mile delivery data to analyse delays, "
    "vehicle performance, agent efficiency, and regional/product bottlenecks."
)

# Sidebar filters
st.sidebar.header("Filters")
weather_filter = st.sidebar.multiselect("Weather", df["weather"].unique(), df["weather"].unique())
traffic_filter = st.sidebar.multiselect("Traffic", df["traffic"].unique(), df["traffic"].unique())
vehicle_filter = st.sidebar.multiselect("Vehicle", df["vehicle"].unique(), df["vehicle"].unique())
area_filter = st.sidebar.multiselect("Area", df["area"].unique(), df["area"].unique())
category_filter = st.sidebar.multiselect("Category", df["category"].unique(), df["category"].unique())

filtered = df[
    (df["weather"].isin(weather_filter)) &
    (df["traffic"].isin(traffic_filter)) &
    (df["vehicle"].isin(vehicle_filter)) &
    (df["area"].isin(area_filter)) &
    (df["category"].isin(category_filter))
].copy()

# KPIs
col1, col2, col3 = st.columns(3)
col1.metric("Avg Delivery Time (mins)", f"{filtered['delivery_time'].mean():.2f}")
col2.metric("Avg Agent Rating", f"{filtered['agent_rating'].mean():.2f}")
col3.metric("Late Deliveries (%)", f"{filtered['late'].mean() * 100:.2f}")

# 1) Delay Analyzer (Weather × Traffic)
st.subheader("⏳ Delay Analyzer – Weather vs Traffic")
delay_df = filtered.groupby(["weather", "traffic"], as_index=False)["delivery_time"].mean()
fig_delay = px.bar(
    delay_df,
    x="traffic",
    y="delivery_time",
    color="weather",
    barmode="group",
    labels={"delivery_time": "Avg Delivery Time (mins)", "traffic": "Traffic"},
    title="Average Delivery Time by Weather and Traffic"
)
st.plotly_chart(fig_delay, use_container_width=True)

# 2) Vehicle Comparison
st.subheader("🚗 Vehicle Comparison")
vehicle_df = filtered.groupby("vehicle", as_index=False)["delivery_time"].mean()
fig_vehicle = px.bar(
    vehicle_df,
    x="vehicle",
    y="delivery_time",
    labels={"delivery_time": "Avg Delivery Time (mins)", "vehicle": "Vehicle Type"},
    title="Average Delivery Time by Vehicle Type"
)
st.plotly_chart(fig_vehicle, use_container_width=True)

# 3) Agent Performance
st.subheader("👤 Agent Performance")
fig_agent = px.scatter(
    filtered,
    x="agent_rating",
    y="delivery_time",
    color="age_group",
    hover_data=["agent_age", "vehicle", "area"],
    labels={"agent_rating": "Agent Rating", "delivery_time": "Delivery Time (mins)"},
    title="Agent Rating vs Delivery Time (by Age Group)",
)
st.plotly_chart(fig_agent, use_container_width=True)

# 4) Area Analysis
st.subheader("📍 Area Bottleneck Analysis")
area_df = filtered.groupby("area", as_index=False)["delivery_time"].mean()
fig_area = px.bar(
    area_df,
    x="area",
    y="delivery_time",
    labels={"delivery_time": "Avg Delivery Time (mins)", "area": "Area"},
    title="Average Delivery Time by Area"
)
st.plotly_chart(fig_area, use_container_width=True)

# 5) Category Visualizer
st.subheader("📦 Category Visualizer")
fig_category = px.box(
    filtered,
    x="category",
    y="delivery_time",
    labels={"delivery_time": "Delivery Time (mins)", "category": "Category"},
    title="Delivery Time Distribution by Product Category"
)
st.plotly_chart(fig_category, use_container_width=True)


Overwriting app.py


In [ ]:
import time, requests, subprocess

!pkill ngrok || echo "no old ngrok process"

proc = subprocess.Popen(["ngrok", "http", "8501"])
time.sleep(5)

tunnels = requests.get("http://localhost:4040/api/tunnels").json()
public_url = tunnels['tunnels'][0]['public_url']
print("🌐 Your public Streamlit URL:", public_url)




🌐 Your public Streamlit URL: https://unpuckered-unforgivingly-alex.ngrok-free.dev
